# Código de configuración inicial

In [50]:
import math
import folium
import random

def haversine(lat1, lon1, lat2, lon2):
    # Radio de la Tierra en kilómetros (aproximado)
    radio_tierra = 6371.0

    # Convertir grados a radianes
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Diferencia de latitud y longitud
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Fórmula de Haversine
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calcular la distancia
    distancia = radio_tierra * c

    return distancia

def calcular_col(fila):
    return haversine(fila['Source Airport Latitude'],fila['Source Airport Longitude'],fila['Destination Airport Latitude'],fila['Destination Airport Longitude'],)

def buscar_datos(codigo):

    fila = df[df['Source Airport Code'] == codigo]

    if not fila.empty:
        primera_fila = fila.iloc[0]
        aux = [primera_fila[0],primera_fila[1],primera_fila[2],primera_fila[3],primera_fila[4],primera_fila[5]]
        return aux
    else:
        fila = df[df['Destination Airport Code'] == codigo]

        if not fila.empty:
            primera_fila = fila.iloc[0]
            aux = [primera_fila[6],primera_fila[7],primera_fila[8],primera_fila[9],primera_fila[10],primera_fila[11]]
            return aux
        else:
            print("No se encontraron resultados que cumplan con la condición.")
            return []

def mostrar_aeropuertos_en_mapa(diccionario_aeropuertos):
    # Crea un mapa
    mapa = folium.Map(location=[0, 0], zoom_start=2)  # Ubicación y zoom iniciales del mapa
    colores = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'darkblue', 'darkgreen', 'pink', 'cadetblue'] * 326
    # Itera a través del diccionario de aeropuertos
    i=0
    for codigo_aeropuerto, coordenadas in diccionario_aeropuertos.items():

        latitud = coordenadas[4]
        longitud = coordenadas[5]
        # Agrega un marcador al mapa para cada aeropuerto
        folium.Marker(
            location=[latitud, longitud],
            popup=coordenadas ,
            tooltip=codigo_aeropuerto,
            icon=folium.Icon(color=colores[i])
        ).add_to(mapa)
        i+=1
    # Muestra el mapa
    mapa.save('mapa_aeropuertos.html')  # Guarda el mapa en un archivo HTML
    return mapa
def mostrar_viajes(camino):
    # Crea un mapa
    mapa = folium.Map(location=[0, 0], zoom_start=2)  # Ubicación y zoom iniciales del mapa
    viajes=[]
    for elem in camino:
        viajes.append(dic_info[elem])
    colores = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'darkblue', 'darkgreen', 'pink', 'cadetblue'] * 2
    # Itera a través del diccionario de aeropuertos y agrega marcadores

    for cod in viajes:
        latitud = cod[4]
        longitud = cod[5]
        num = random.randint(0, 19)
        folium.Marker(
            location=[latitud, longitud],
            popup= f'{cod}',
            tooltip= cod[0], icon=folium.Icon(color=colores[num])
        ).add_to(mapa)

    for i in range(1,len(viajes)):
        num = random.randint(0, 19)
        folium.PolyLine(
                locations=[[viajes[i-1][4],viajes[i-1][5]],[viajes[i][4],viajes[i][5]]],
                color=colores[num],
                weight=2.5
            ).add_to(mapa)
        pass

    # Muestra el mapa
    mapa.save('mapa_aeropuertos_con_vuelos.html')
    return mapa


def mostrar_mapa_nodo(nodos,inicial):
    mapa = folium.Map(location=[0, 0], zoom_start=2)  # Ubicación y zoom iniciales del mapa
    viajes = []
    inicial = dic_info[inicial]
    for elem in nodos:
        viajes.append(dic_info[elem[0]])
    colores = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'darkblue', 'darkgreen', 'pink', 'cadetblue'] * 2
    # Itera a través del diccionario de aeropuertos y agrega marcadores
    i = 0
    folium.Marker(location=[inicial[4],inicial[5]],
            popup= inicial,
            tooltip= inicial[0], icon=folium.Icon(color=colores[7])).add_to(mapa)
    for cod in viajes:
        latitud = cod[4]
        longitud = cod[5]
        num = random.randint(0, 19)
        folium.Marker(
            location=[latitud, longitud],
            popup= f'-{cod}',
            tooltip= nodos[i], icon=folium.Icon(color=colores[num])
        ).add_to(mapa)
        i+=1

    # Muestra el mapa
    mapa.save('mapa_nodo_solo.html')

    return mapa


In [43]:
import pandas as pd
df = pd.read_csv('flights_final.csv')
df = df.drop_duplicates()

df['Combinacion'] = df.apply(lambda row: ''.join(sorted([row['Source Airport Code'], row['Destination Airport Code']])), axis=1)
conteo_vuelos_reciprocos = df.groupby('Combinacion').size().reset_index(name='Cantidad de Vuelos Recíprocos')
combinaciones_reciprocas = conteo_vuelos_reciprocos[conteo_vuelos_reciprocos['Cantidad de Vuelos Recíprocos'] > 1]

print(combinaciones_reciprocas)
df = df.sort_values(['Source Airport Code', 'Destination Airport Code'])
df_sin_duplicados = df.drop_duplicates(subset='Combinacion', keep='first').drop(columns='Combinacion')
df_final = df_sin_duplicados.copy()
df_final['Distancia'] = df.apply(calcular_col, axis = 1)
df_final

      Combinacion  Cantidad de Vuelos Recíprocos
0          AAEALG                              2
1          AAECDG                              2
2          AAEIST                              2
3          AAELYS                              2
4          AAEMRS                              2
...           ...                            ...
18923      ZADZAG                              2
18924      ZAGZRH                              2
18925      ZAHZBR                              2
18927      ZGSZKG                              2
18928      ZLTZTB                              2

[18109 rows x 2 columns]


,Source Airport Code,Source Airport Name,Source Airport City,Source Airport Country,Source Airport Latitude,Source Airport Longitude,Destination Airport Code,Destination Airport Name,Destination Airport City,Destination Airport Country,Destination Airport Latitude,Destination Airport Longitude,Distancia
1804,AAE,Rabah Bitat Airport,Annaba,Algeria,36.822201,7.809174,ALG,Houari Boumediene Airport,Algier,Algeria,36.691002,3.215410,409.468297
21895,AAE,Rabah Bitat Airport,Annaba,Algeria,36.822201,7.809174,CDG,Charles de Gaulle International Airport,Paris,France,49.012798,2.550000,1420.677874
49359,AAE,Rabah Bitat Airport,Annaba,Algeria,36.822201,7.809174,IST,Istanbul Airport,Istanbul,Turkey,41.275278,28.751944,1869.884346
31895,AAE,Rabah Bitat Airport,Annaba,Algeria,36.822201,7.809174,LYS,Lyon Saint-Exupéry Airport,Lyon,France,45.725556,5.081111,1015.746821
5205,AAE,Rabah Bitat Airport,Annaba,Algeria,36.822201,7.809174,MRS,Marseille Provence Airport,Marseille,France,43.439272,5.221424,767.859518
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54510,ZRH,Zürich Airport,Zurich,Switzerland,47.464699,8.549170,ICN,Incheon International Airport,Seoul,South Korea,37.469101,126.450996,8743.102456
50460,ZSA,San Salvador Airport,Cockburn Town,Bahamas,24.063299,-74.524002,PLS,Providenciales Airport,Providenciales,Turks and Caicos Islands,21.773600,-72.265900,343.941602
39013,ZSJ,Sandy Lake Airport,Sandy Lake,Canada,53.064201,-93.344398,KEW,Keewaywin Airport,Keewaywin,Canada,52.991100,-92.836403,34.931482
42156,ZSJ,Sandy Lake Airport,Sandy Lake,Canada,53.064201,-93.344398,YPM,Pikangikum Airport,Pikangikum,Canada,51.819698,-93.973297,144.797504


In [44]:
nodos = df_final.groupby('Source Airport Code').size().reset_index(name='Cantidad de Vuelos')
print(nodos)


     Source Airport Code  Cantidad de Vuelos
0                    AAE                   7
1                    AAL                  13
2                    AAN                   2
3                    AAQ                   3
4                    AAR                   7
...                  ...                 ...
2392                 ZQW                   1
2393                 ZRH                   1
2394                 ZSA                   1
2395                 ZSJ                   2
2396                 ZTH                   1

[2397 rows x 2 columns]


In [45]:
Source_air = df_final['Source Airport Code'].tolist()
Source_air = list(set(Source_air))
Source_air.sort()
Destination_air = df_final['Destination Airport Code'].tolist()
Destination_air = list(set(Destination_air))
Destination_air.sort()
aux = []
faltan = []
for elem in Source_air:
    if not(elem in aux):
        aux.append(elem)
    else:
        pass
for elem in Destination_air:
    if not(elem in aux):
        aux.append(elem)
        faltan.append(elem)
    else:
        pass

aux.sort()
dic = {valor: indice for indice, valor in enumerate(aux)}
dic_inv = {valor: clave for clave, valor in dic.items()}
dic_info = {dato: buscar_datos(dato) for dato in aux}
print(len(dic))
dic_faltan = {}
for airport in faltan:
    fila = df_final[df_final['Destination Airport Code'] == airport]
    if not fila.empty:
        fila_info = fila[['Destination Airport Latitude', 'Destination Airport Longitude','Destination Airport Name', 'Destination Airport City', 'Destination Airport Country']].iloc[0]
        dic_faltan[airport] = fila_info.to_list()


C:\Users\Gow14\AppData\Local\Temp\ipykernel_26336\2889623129.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  aux = [primera_fila[0],primera_fila[1],primera_fila[2],primera_fila[3],primera_fila[4],primera_fila[5]]
C:\Users\Gow14\AppData\Local\Temp\ipykernel_26336\2889623129.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  aux = [primera_fila[6],primera_fila[7],primera_fila[8],primera_fila[9],primera_fila[10],primera_fila[11]]


3256


# Clase del grafo ponderado

In [46]:
from typing import List
class WeightedGraph:

    INF = (1 << 31) - 1  # 2 ** 31 - 1

    def __init__(self, n: int) -> None:
        self.n = n
        self.C = [[0 for i in range(n)] for j in range(n)]

    def add_edge(self, vi: int, vf: int, weight: int = 1) -> bool:
        if not ((0 <= vi < self.n) and (0 <= vf < self.n)):
            return False
        self.C[vi][vf] = self.C[vf][vi] = weight
        return True

    def remove_edge(self, vi: int, vf: int) -> bool:
        if not ((0 <= vi < self.n) and (0 <= vf < self.n)):
            return False
        self.C[vi][vf] = self.C[vf][vi] = 0
        return True

    def dijkstra(self, nodo: 'str'):

        def all_visit(visit: List[bool]) -> bool:
            for v in visit:
                if not v:
                    return False
            return True

        def min_dist_not_visit(D: List[int], visit: List[bool]) -> int:
            min_dist, index = WeightedGraph.INF + 1, -1
            for i in range(self.n):
                if D[i] < min_dist and not visit[i]:
                    min_dist = D[i]
                    index = i
            return index

        v0 = dic[nodo]  # Transformar el nodo a numero

        D = [WeightedGraph.INF] * self.n
        pad = [None] * self.n
        visit = [False] * self.n

        D[v0] = 0

        while not all_visit(visit):
            v = min_dist_not_visit(D, visit)
            visit[v] = True
            for i in range(self.n):
                if self.C[v][i] != 0 and D[v] + self.C[v][i] < D[i] and not visit[i]:   # if es adyacente and mejora el camino and not visit
                    D[i] = D[v] + self.C[v][i]
                    pad[i] = v

        return D, pad
        #return {dic_inv[index]: (dist, dic_inv[p] if p is not None else p) for index, dist, p in zip(range(self.n), D, pad)}
        #return self.caminos_Max10(D,pad)

    def __caminos_Max10(self, dist:list , padres:list ):
        enumer_dist = list(enumerate(dist))
        valor_referencia = (1 << 31) - 1
        dist_rest = [(indice, distancia) for indice, distancia in enumer_dist if distancia < valor_referencia]
        dist_rest.sort(key=lambda x: x[1], reverse=True)
        diez_mayores_dist = dist_rest[:10]
        caminos_max10 = []
        for elem in diez_mayores_dist:
            caminos_max10.append([dic_inv[elem[0]],dist[elem[0]],dic_inv[padres[elem[0]]]])
        return caminos_max10

    def dic_dijkstra(self, nodo:str):
        dis , pad = self.dijkstra(nodo)
        return {dic_inv[index]: (dist, dic_inv[p] if p is not None else p) for index, dist, p in zip(range(self.n), dis, pad)}

    def Caminos_Max(self, nodo:'str' ):
        dist , pad = self.dijkstra(nodo)
        return self.__caminos_Max10(dist,pad)

    def camino_min(self, nodo1:str, nodo2_s:str):
        dist , pad = self.dijkstra(nodo1)
        nodo2 = dic[nodo2_s]
        boole = True
        camino = []
        if(pad[nodo2] is not None):
            padre = dic_inv[pad[nodo2]]
            while(boole):
                camino.append(padre)

                if(padre== nodo1):
                    boole = False
                else:
                    padre = dic_inv[pad[dic[padre]]]
        else:
            if(nodo1 == nodo2):
                print("Es el mismo lugar")
            else:
                print('No tiene un camino hacia ese lugar')
                return [],None
        camino.reverse()
        camino.append(nodo2_s)
        return camino, dist[nodo2]

    def floyd_warshall(self):
        D = [row.copy() for row in self.C]
        for i in range(self.n):
            for j in range(self.n):
                if D[i][j] == 0 and i != j:
                    D[i][j] = WeightedGraph.INF

        path = [[i for i in range(self.n)] for j in range(self.n)]
        for i in range(self.n):
            path[i][i] = None
        for k in range(self.n):
            for i in range(self.n):
                for j in range(self.n):
                    if D[i][k] + D[k][j] < D[i][j]:
                        D[i][j] = D[i][k] + D[k][j]
                        path[i][j] = k
        return D, path

    def nodo_indvidual(self, nodo: str):
        print(dic_info[nodo],'\n')
        camin = self.Caminos_Max(nodo)
        aereoMax = []
        for elem in camin:
            print(dic_info[elem[0]], " Y la distancia es de ",elem[1])
            aereoMax.append([elem[0],elem[1]])

        return mostrar_mapa_nodo(aereoMax, nodo)

    def nodo_duo(self, nodo1:str, nodo2:str ):
        cam , distan = self.camino_min(nodo1,nodo2)
        # mostrar_viajes(cam)
        for elem in cam:
            print('Pasa por ', dic_info[elem])
        if(distan is not None):
            print('Y la distancia es de',distan)
        return mostrar_viajes(cam)

In [28]:
grafo = WeightedGraph(len(dic))
for i, fila in df_final.iterrows():
    grafo.add_edge(dic[fila['Source Airport Code']],dic[fila['Destination Airport Code']],fila['Distancia'])

In [ ]:
mostrar_aeropuertos_en_mapa(dic_info) # Muestra todos los aeropuertos en el mapa (¡NO EJECUTAR!)

# Tareas

In [39]:
# Número 1: 10 aeropuertos con los caminos mínimos más largos
aeropuerto1 = 'BOG'
grafo.nodo_indvidual(aeropuerto1)

['BOG', 'El Dorado International Airport', 'Bogota', 'Colombia', 4.70159, -74.1469] 

['MKQ', 'Mopah Airport', 'Merauke', 'Indonesia', -8.520290375, 140.4179993]  Y la distancia es de  22748.08548446928
['CCK', 'Cocos (Keeling) Islands Airport', 'Cocos Keeling Island', 'Cocos (Keeling) Islands', -12.18830013, 96.83390045]  Y la distancia es de  22465.401252602267
['OKL', 'Oksibil Airport', 'Oksibil', 'Indonesia', -4.9071, 140.6277]  Y la distancia es de  22346.52430515729
['WMX', 'Wamena Airport', 'Wamena', 'Indonesia', -4.10251, 138.957001]  Y la distancia es de  22329.45664483206
['DJJ', 'Sentani International Airport', 'Jayapura', 'Indonesia', -2.576950073, 140.5160065]  Y la distancia es de  22087.127269088927
['TIM', 'Moses Kilangin Airport', 'Timika', 'Indonesia', -4.528279781, 136.8869934]  Y la distancia es de  21696.726235460676
['BIK', 'Frans Kaisiepo Airport', 'Biak', 'Indonesia', -1.190019965, 136.1080017]  Y la distancia es de  21662.63788984528
['NBX', 'Nabire Airport', '

In [52]:
# Número 1: aeropuerto aleatorio
air1 = random.choice(list(dic.keys()))
grafo.nodo_indvidual(air1)

['CLJ', 'Cluj-Napoca International Airport', 'Cluj-napoca', 'Romania', 46.78519821, 23.68619919] 

['IUE', 'Niue International Airport', 'Alofi', 'Niue', -19.07903099, -169.9255981]  Y la distancia es de  20289.558801501593
['GMR', 'Totegegie Airport', 'Totegegie', 'French Polynesia', -23.07990074, -134.8899994]  Y la distancia es de  18700.795584676853
['UAH', 'Ua Huka Airport', 'Ua Huka', 'French Polynesia', -8.93611, -139.552002]  Y la distancia es de  18530.96102728976
['UAP', 'Ua Pou Airport', 'Ua Pou', 'French Polynesia', -9.35167, -140.078003]  Y la distancia es de  18518.974671430187
['AUQ', 'Hiva Oa-Atuona Airport', 'Hiva-oa', 'French Polynesia', -9.768790245, -139.0110016]  Y la distancia es de  18483.35892274695
['NHV', 'Nuku Hiva Airport', 'Nuku Hiva', 'French Polynesia', -8.795599937, -140.2290039]  Y la distancia es de  18454.958068843825
['TIU', 'Timaru Airport', 'Timaru', 'New Zealand', -44.30279922, 171.2250061]  Y la distancia es de  18375.610168320058
['MUK', 'Mauke 

In [53]:
# Número 2: Camino mínimo entre dos aeropuertos y sus aeropuertos intermedios
grafo.nodo_duo('COK','AAE')

Pasa por  ['COK', 'Cochin International Airport', 'Kochi', 'India', 10.152, 76.401901]
Pasa por  ['MCT', 'Muscat International Airport', 'Muscat', 'Oman', 23.59329987, 58.28440094]
Pasa por  ['IST', 'Istanbul Airport', 'Istanbul', 'Turkey', 41.275278, 28.751944]
Pasa por  ['AAE', 'Rabah Bitat Airport', 'Annaba', 'Algeria', 36.822201, 7.809174]
Y la distancia es de 7674.942781330655


In [55]:
# Número 2: aeropuertos aleatorios
air1, air2 = random.choice(list(dic.keys())), random.choice(list(dic.keys()))
print(f'Ruta de {air1} a {air2}:')
grafo.nodo_duo(air1, air2)

Ruta de MSU a TNW:
Pasa por  ['MSU', 'Moshoeshoe I International Airport', 'Maseru', 'Lesotho', -29.46229935, 27.55249977]
Pasa por  ['JNB', 'OR Tambo International Airport', 'Johannesburg', 'South Africa', -26.1392, 28.246]
Pasa por  ['GRU', 'Guarulhos - Governador André Franco Montoro International Airport', 'Sao Paulo', 'Brazil', -23.43555641, -46.47305679]
Pasa por  ['UIO', 'Mariscal Sucre International Airport', 'Quito', 'Ecuador', -0.129166667, -78.3575]
Pasa por  ['TNW', 'Jumandy Airport', 'Tena', 'Ecuador', -1.059722, -77.583333]
Y la distancia es de 12257.560928830519
